In [1]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns
import datetime
from datetime import date, timedelta

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix

import pylab as pl
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import pickle

from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import export_graphviz 
    #In the cmd line type: conda install python-graphviz
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("no_show_original_data_2016_with_update_weather.csv") 
df=df.drop(['Unnamed: 0'], axis=1)

In [3]:
#ScheduledDay קיצוץ התאריך של והופך את התאריך לסוג דייטיים
df['ScheduledDay'] =df['ScheduledDay'].apply(str)
df['ScheduledDay'] = df['ScheduledDay'].str.slice(0, 10, 1) 
df['ScheduledDay'] =  pd.to_datetime(df['ScheduledDay'], format='%Y-%m-%d')

In [4]:
# הפיכת שדות מז"א למספריים
df['prcp']=df['prcp'].astype(float).astype(int)
df['temp']=df['temp'].astype(float).astype(int)

In [5]:
#AppointmentDay הופך את התאריך לסוג דייטיים
df['AppointmentDay'] =df['AppointmentDay'].apply(str)
df['AppointmentDay'] = df['AppointmentDay'].str.slice(0, 10, 1) 
df['AppointmentDay'] =  pd.to_datetime(df['AppointmentDay'], format='%Y-%m-%d')

In [6]:
# הכנסת קודים לשכונות
df.loc[:, 'Neighbourhood_codes'] = df["Neighbourhood"].astype("category").cat.codes+1
# הפיכת שדות קטגוריים לבינריים
df.loc[:, 'Gender_'] = df["Gender"].astype("category").cat.codes
#Woman=0, Man=1
df.loc[:, 'No-show_'] = df["No-show"].astype("category").cat.codes
#show=0, no-show=1 

In [7]:
#הוספת שדה של יום בשבוע של הבדיקה
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['Appointment_Week_Day'] = df['AppointmentDay'].dt.dayofweek.map(dayOfWeek)

In [8]:
#הוספת שדה של המרווח בין הבדיקות
df['Delta_Dates'] = (df['AppointmentDay']-df['ScheduledDay']).dt.days
# בדיקות שתואמו אחרי הבדיקה עצמה יופיעו כמינוס 1-כי לא הגיוני
mask = df.Delta_Dates < 0
column_name = 'Delta_Dates'
df.loc[mask, column_name] = -1

In [9]:
# חיתוך כל מי שיש לו דלתא קטנה מ1
#df[df.Delta_Dates < 1] מוחק
#38562 רשומות של פגישות שנקבעו באותו תאריך
# 5 רשומות של דלתא שלילית=לא הגיוני
df = df.drop(df[df.Delta_Dates < 1].index)

In [10]:
#Get Dummies לימי השבוע של הפגישה
week_day=pd.get_dummies(df["Appointment_Week_Day"])
week_day["AppointmentID"] = df["AppointmentID"]
df=df.merge(week_day, how='left', on='AppointmentID')
df = df.drop(['Appointment_Week_Day'], axis=1)

In [11]:
# מוסיף עמודה של כמות פגישות לכל מטופל
a=df.groupby(['PatientId']).size().reset_index(name='Appointment_counts')
df=df.merge(a, how='left', on='PatientId')
#בדיקה עצמית לקוד
#df.loc[df['PatientId'] == 1.885170e+14]

In [12]:
#Get Dummies לשכונות
NC=pd.get_dummies(df["Neighbourhood_codes"])
NC["AppointmentID"] = df["AppointmentID"]
df=df.merge(NC, how='left', on='AppointmentID')

In [13]:
df["yesterday"] = df["AppointmentDay"] - timedelta(1)

In [15]:
df.columns

Index([     'PatientId',  'AppointmentID',         'Gender',   'ScheduledDay',
       'AppointmentDay',            'Age',  'Neighbourhood',    'Scholarship',
         'Hipertension',       'Diabetes',
       ...
                     73,               74,               75,               76,
                     77,               78,               79,               80,
                     81,      'yesterday'],
      dtype='object', length=109)

In [21]:
# הורדת שדות מיותרים
#df = df.drop(columns=['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay', 'No-show', 'Neighbourhood', 'Neighbourhood_codes','AppointmentDay'], axis = 1)

In [30]:
df.columns

Index([     'PatientId',  'AppointmentID',         'Gender',   'ScheduledDay',
       'AppointmentDay',            'Age',  'Neighbourhood',    'Scholarship',
         'Hipertension',       'Diabetes',
       ...
                     73,               74,               75,               76,
                     77,               78,               79,               80,
                     81,      'yesterday'],
      dtype='object', length=109)

new_index_order = ['Age','Gender_','Delta_Dates','SMS_received', 'Friday', 'Monday', 'Saturday', 'Thursday', 'Tuesday', 'Wednesday','Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism',
       'Handcap','No-show_']
df = df.reindex(columns=new_index_order)

In [31]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,73,74,75,76,77,78,79,80,81,yesterday
0,2.366230e+14,5628286,M,2016-04-27,2016-04-29,0,SÃO BENEDITO,0,0,0,...,0,0,1,0,0,0,0,0,0,2016-04-28
1,1.885170e+14,5616082,M,2016-04-25,2016-04-29,0,ILHA DAS CAIEIRAS,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-04-28
2,2.718820e+14,5628321,M,2016-04-27,2016-04-29,0,CONQUISTA,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-04-28
3,3.326990e+12,5632495,F,2016-04-28,2016-04-29,0,SANTO ANTÔNIO,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-04-28
4,8.468590e+14,5560742,M,2016-04-08,2016-04-29,0,PRAIA DO SUÁ,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-04-28


In [137]:
pickle.dump(df, open("df_c.dat", "wb"))